# Data processing notebook
author: Gonzalo Miranda Cabrera

objective: create a clean dataset with the provided data tables for later training of ml models.

summary:
1. cleaning tables: process each table and get it ready for a join into one dataset.
2. join tables: combine the tables into one.
3. outliers: train an isolation forest for anomaly detection.
4. features: add more features to the dataset. (correlation matrix and rates between preexisting columns)
5. Split Nan for submission: save rows with total_minutes with nan values for submission file.
6. write csv: save data and submission to disk.


In [ ]:
import numpy as np
import pandas as pd
from seaborn import heatmap
import matplotlib.pyplot as plt
from geopy.distance import distance
from sklearn.ensemble import IsolationForest


In [ ]:
order_products = pd.read_csv('data/order_products.csv')
orders = pd.read_csv('data/orders.csv')
shoppers = pd.read_csv('data/shoppers.csv')
storebranch = pd.read_csv('data/storebranch.csv')

## Cleaning tables

### order_products table


In [ ]:
order_products.head()

With the provided orders_preducts table we can extract the following:
- total units per order
- total kg per order
- total unique products per order

In [ ]:
# Aggregate total units and total kgs per order
total_order_products = (
    order_products.groupby(["order_id", "buy_unit"]).sum().reset_index()
)
total_order_products.head(7)


In [ ]:
# Calculate the amount of unique products per order
unique_products = (
    order_products.groupby(["order_id"])['product_id'].count().reset_index()
)
unique_products.head(7)

In [ ]:
# Leave only unique order_id on order_products table
order_products.drop_duplicates(subset=['order_id'], inplace=True)
order_products.drop(columns=['product_id', 'quantity', 'buy_unit'], inplace=True)
order_products.head()

In [ ]:
# Add unique products per order to order_products table
order_products = (
    order_products.set_index("order_id")
    .join(unique_products.set_index("order_id"))
    .reset_index()
)
order_products.rename(columns={"product_id": "unique_products"}, inplace=True)
order_products.head()

In [ ]:
# Add total units per order_id
filter_ = total_order_products["buy_unit"] == "UN"
order_products = (
    order_products.set_index("order_id")
    .join(total_order_products[filter_][["order_id", "quantity"]].set_index("order_id"))
    .reset_index()
)
order_products.rename(columns={"quantity": "units"}, inplace=True)
order_products.head()


In [ ]:
# Add total kgs per order_id
filter_ = total_order_products["buy_unit"] == "KG"
order_products = (
    order_products.set_index("order_id")
    .join(total_order_products[filter_][["order_id", "quantity"]].set_index("order_id"))
    .reset_index()
)
order_products.rename(columns={"quantity": "kgs"}, inplace=True)
order_products.head(30)


In [ ]:
# Fill NaN values of order_products table
order_products.fillna(0, inplace=True)

del filter_, total_order_products, unique_products

order_products.head(30)


### Orders table

In [ ]:
orders.head()

In the orders table we can do the following:
- obtain day of the week that the order took place from promised_time
- convert lat and lng to a point to calculate distance with store
- convert on_demand to int

In [ ]:
orders.isna().sum()

In [ ]:
# Check the days, month and years that the orders took place
orders['promised_time'] = pd.to_datetime(orders['promised_time'])
orders.promised_time.dt.day.unique(), orders.promised_time.dt.month.unique(), orders.promised_time.dt.year.unique()

In [ ]:
# Create one-hot encoded columns for the days
orders['day'] = orders.promised_time.dt.day_name()
orders = pd.concat(
    [orders, pd.get_dummies(orders.day, prefix="is")], axis=1
)
orders.drop(columns=['promised_time', 'day'], inplace=True)

In [ ]:
# Create delivery_point for distance calculation with stores
orders["delivery_point"] = list(zip(orders.lat, orders.lng))
orders.drop(columns=["lat", "lng"], inplace=True)

# Covnert bool to int
orders["on_demand"] = orders["on_demand"].astype(int)
orders.head()


### Shoppers table

In [ ]:
shoppers.head()


from the shoppers table we can do:
- change seniority to one-hot encoding
- fill NaN with mean

In [ ]:
shoppers.isna().sum()

In [ ]:
# Fill NaN with mean of each column
shoppers.fillna(shoppers.mean(), inplace=True)

In [ ]:
shoppers.isna().sum()

In [ ]:
# Convert seniority categorical to one hot encoding
shoppers = pd.concat(
    [shoppers, pd.get_dummies(shoppers.seniority, prefix="seniority")], axis=1
)
shoppers.drop(columns=["seniority"], inplace=True)

shoppers.head()


### Storebranch table

In [ ]:
storebranch.head()

In [ ]:
# Create store_point for distance calculation with orders
storebranch["store_point"] = list(zip(storebranch["lat"], storebranch["lng"]))
storebranch.drop(columns=["lat", "lng", "store_id"], inplace=True)
storebranch.head()


## Join tables

We join all the above tables into the data variable

In [ ]:
orders.head()

In [ ]:
# Join orders and storebranch tables into data
data = (
    orders.set_index("store_branch_id")
    .join(storebranch.set_index("store_branch_id"))
    .reset_index()
)
data.drop(columns=["store_branch_id"], inplace=True)
data.head()


In [ ]:
# Calculate distance from delivery_point to store_point
data["distance"] = data[["delivery_point", "store_point"]].apply(
    lambda values: distance(values[0], values[1]).kilometers, axis=1
)
data.drop(columns=["delivery_point", "store_point"], inplace=True)
data.head()


In [ ]:
# Join data and shoppers tables
data = (
    data.set_index("shopper_id").join(shoppers.set_index("shopper_id")).reset_index()
)
data.drop(columns=["shopper_id"], inplace=True)
data.head()


In [ ]:
# Join data and order_products tables
data = data.set_index("order_id").join(order_products.set_index("order_id"))
data.head()


In [ ]:
# Check for NaN values
data.isna().sum()

In [ ]:
# order_ids in orders but not in order_products
data.drop(
    data[
        (data["units"].isna() | data["kgs"].isna()) | data["unique_products"].isna()
    ].index,
    inplace=True,
)
data.isna().sum()


In [ ]:
# show data
data

In [ ]:
# delete unused variables
del storebranch, shoppers, order_products, orders

## Outliers

###  Split nan for isolation forest training
as isolation forest can not be trained if dataset has NaN values.

In [ ]:
submission = data[pd.isna(data['total_minutes'])]
data.dropna(inplace=True)
data.shape, submission.shape

### Isolation forest training

In [ ]:
isolation_forest = IsolationForest(
    max_features=1.0,
    contamination=0.05,
    n_jobs=-1,
    random_state=0,
)

data["anomaly_label"] = isolation_forest.fit_predict(data)
data[data["anomaly_label"] == -1]


### Drop found anomalies
anomalies are set with a label of -1, so we select all the labels that are 1.

In [ ]:
data = data[data["anomaly_label"] == 1]
data = data.drop(columns=['anomaly_label'])

### Merge data for features

In [ ]:
data = pd.concat([data, submission])
data.isna().sum()


## Features

to make more features we use the correlation matrix (pearson) to find the most correlated with total_minutes

In [ ]:
# Plot heatmap
plt.figure(figsize=(20, 20))
heatmap(
    data.corr(), annot=True, linewidth=0.8, mask=np.triu(data.corr()), cmap="RdYlBu_r"
)


unique_products, units and kgs are the features that have greater correlation with total_minutes.

so we create the following features:


In [ ]:
# create estimated order size assuming 1 kg is 1 unit
data["order_size"] = data.units + data.kgs

# scale most correlated features with math functions
data["root_order_size"] = np.sqrt(data.order_size)
data["sqrd_order_size"] = data.order_size ** 2
data["logn_order_size"] = np.log(data.order_size)

data["root_units"] = np.sqrt(data.units)
data["sqrd_units"] = data.units ** 2
data["logn_units"] = np.log(data.units + 1)

data["root_kgs"] = np.sqrt(data.kgs)
data["sqrd_kgs"] = data.kgs ** 2
data["logn_kgs"] = np.log(data.kgs + 1)

data["root_unique_products"] = np.sqrt(data.unique_products)
data["sqrd_unique_products"] = data.unique_products ** 2
data["logn_unique_products"] = np.log(data.unique_products)

# Create different rates with distance as it has more correlation than other features
data["distance_div_units"] = np.true_divide(data.distance, data.units + 1)
data["distance_div_kgs"] = np.true_divide(data.distance, data.kgs + 1)
data["distance_div_unique_products"] = data.distance / data.unique_products
data["distance_div_order_size"] = data.distance / data.order_size

# Create different rates with diferent amounts and picking speed that make a sense of time.
data["unique_products_div_picking_speed"] = data.unique_products / data.picking_speed
# For example we are dividing products by products/minute and we end up with minutes.

data["units_div_picking_speed"] = data.units / data.picking_speed
data["kgs_div_picking_speed"] = data.kgs / data.picking_speed
data["order_size_div_picking_speed"] = data.order_size / data.picking_speed


In [ ]:
# plot the heatmap again to see the correlation of the newly added features
plt.figure(figsize=(30, 30))
heatmap(
    data.corr(), annot=True, linewidth=0.8, mask=np.triu(data.corr()), cmap="RdYlBu_r"
)


In [ ]:
# Show data.min and data.max to see if there are some undefined values
data.min(), data.max()

no undefined values are found

## Split NaN for later submission

In [ ]:
submission = data[pd.isna(data['total_minutes'])]
data.dropna(inplace=True)
data.shape, submission.shape


## Write csv

In [ ]:
# Save data and submission to disk
data.to_csv('data.csv')
submission.to_csv('submission_data.csv')